## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [HCC Sentence entity resolver](https://aws.amazon.com/marketplace/pp/prodview-tlv5sqqbdzwzy)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline for Hierarchical Condition Categories (HCC) Sentence Entity Resolver

- **Model**: `hcc_vdb_resolver`
- **Model Description**: This pretrained pipeline extracts clinical entities from clinical text and maps them to their corresponding Hierarchical Condition Categories (HCC) codes.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse

In [ ]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
model_name = "hcc-vdb-resolver"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [8]:
docs = [
    """The patient presents with a history of Hypertension and Hyperlipidemia, which have been managed with medication. Recent lab results indicate elevated cholesterol levels, and the patient reports occasional chest pain.""", 
    """Upon examination, the patient was found to have signs of Congestive Heart Failure and Atrial Fibrillation. The patient's condition requires immediate attention and an adjustment of their current treatment plan.""",
]


sample_text = """The medical record of the patient indicates a diagnosis of Diabetes and Chronic Obstructive Pulmonary Disease, requiring comprehensive care and management."""

### JSON

In [9]:
input_json_data = {"text": sample_text}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
pd.DataFrame(response_json["predictions"][0])

,begin,end,ner_chunk,ner_label,ner_confidence,concept_code,resolution,score,all_codes,concept_name_detailed,all_resolutions,all_score
0,59,66,Diabetes,PROBLEM,0.9878,19,diabetes mellitus (disorder),0.839343,"[19, 19, 0, 19, 0]","[diabetes mellitus (disorder) [type 2 diabetes mellitus without complications], diabetes mellitus [type 1 diabetes mellitus without complications], renal diabetes (disorder) [other specified diabetes mellitus with kidney complications], type i diabetes mellitus [type 1 diabetes mellitus without complications], type 1 diabetes mellitus [type 1 diabetes mellitus]]","[diabetes mellitus (disorder), diabetes mellitus, renal diabetes (disorder), type i diabetes mellitus, type 1 diabetes mellitus]","[0.8393428325653076, 0.8375614881515503, 0.8063362836837769, 0.7999697923660278, 0.7912539839744568]"
1,72,108,Chronic Obstructive Pulmonary Disease,PROBLEM,0.947025,111,chronic obstructive pulmonary disease,1.000000,"[111, 111, 111, 111, 111]","[chronic obstructive pulmonary disease [chronic obstructive pulmonary disease, unspecified], chronic obstructive lung disease [chronic obstructive pulmonary disease, unspecified], copd - chronic obstructive pulmonary disease [chronic obstructive pulmonary disease, unspecified], chronic obstructive lung disease (disorder) [chronic obstructive pulmonary disease, unspecified], chronic obstructive airway disease [chronic obstructive pulmonary disease, unspecified]]","[chronic obstructive pulmonary disease, chronic obstructive lung disease, copd - chronic obstructive pulmonary disease, chronic obstructive lung disease (disorder), chronic obstructive airway disease]","[0.9999998807907104, 0.9534466862678528, 0.9277709126472473, 0.9195234179496765, 0.8972678780555725]"


### JSON Lines

In [10]:
def create_jsonl(records):
    if isinstance(records, str):
        records = [records]
    json_records = [{"text": text} for text in records]
    json_lines = "\n".join(json.dumps(record) for record in json_records)
    return json_lines

In [11]:
input_jsonl_data = create_jsonl(sample_text)
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"begin": 59, "end": 66, "ner_chunk": "Diabetes", "ner_label": "PROBLEM", "ner_confidence": "0.9878", "concept_code": "19", "resolution": "diabetes mellitus (disorder)", "score": 0.8393428325653076, "all_codes": ["19", "19", "0", "19", "0"], "concept_name_detailed": ["diabetes mellitus (disorder) [type 2 diabetes mellitus without complications]", "diabetes mellitus [type 1 diabetes mellitus without complications]", "renal diabetes (disorder) [other specified diabetes mellitus with kidney complications]", "type i diabetes mellitus [type 1 diabetes mellitus without complications]", "type 1 diabetes mellitus [type 1 diabetes mellitus]"], "all_resolutions": ["diabetes mellitus (disorder)", "diabetes mellitus", "renal diabetes (disorder)", "type i diabetes mellitus", "type 1 diabetes mellitus"], "all_score": [0.8393428325653076, 0.8375614881515503, 0.8063362836837769, 0.7999697923660278, 0.7912539839744568]}, {"begin": 72, "end": 108, "ner_chunk": "Chronic Obstructive Pulmo

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [15]:
validation_json_file_name = "input.json"
validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [16]:
# Create JSON and JSON Lines data
input_jsonl_data = create_jsonl(docs)
input_json_data = json.dumps({"text": docs})

# Upload JSON and JSON Lines data to S3
upload_to_s3(input_json_data, validation_json_file_name)
upload_to_s3(input_jsonl_data, validation_jsonl_file_name)

### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [18]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [19]:
retrieve_json_output_from_s3(validation_json_file_name)

{'predictions': [[{'begin': 39,
    'end': 50,
    'ner_chunk': 'Hypertension',
    'ner_label': 'PROBLEM',
    'ner_confidence': '0.9841',
    'concept_code': '0',
    'resolution': 'hypertension',
    'score': 0.9999997615814209,
    'all_codes': ['0', '0', '0', '0', '0'],
    'concept_name_detailed': ['hypertension [essential (primary) hypertension]',
     'renal hypertension [chronic kidney disease, unspecified]',
     'high blood pressure [neonatal hypertension]',
     'systolic hypertension [neonatal hypertension]',
     'diastolic hypertension [neonatal hypertension]'],
    'all_resolutions': ['hypertension',
     'renal hypertension',
     'high blood pressure',
     'systolic hypertension',
     'diastolic hypertension'],
    'all_score': [0.9999997615814209,
     0.8857569694519043,
     0.8678925037384033,
     0.8500458002090454,
     0.8453720808029175]},
   {'begin': 56,
    'end': 69,
    'ner_chunk': 'Hyperlipidemia',
    'ner_label': 'PROBLEM',
    'ner_confidence': '0

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [21]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [22]:
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name)

{"predictions": [{"begin": 39, "end": 50, "ner_chunk": "Hypertension", "ner_label": "PROBLEM", "ner_confidence": "0.9841", "concept_code": "0", "resolution": "hypertension", "score": 0.9999997615814209, "all_codes": ["0", "0", "0", "0", "0"], "concept_name_detailed": ["hypertension [essential (primary) hypertension]", "renal hypertension [chronic kidney disease, unspecified]", "high blood pressure [neonatal hypertension]", "systolic hypertension [neonatal hypertension]", "diastolic hypertension [neonatal hypertension]"], "all_resolutions": ["hypertension", "renal hypertension", "high blood pressure", "systolic hypertension", "diastolic hypertension"], "all_score": [0.9999997615814209, 0.8857569694519043, 0.8678925037384033, 0.8500458002090454, 0.8453720808029175]}, {"begin": 56, "end": 69, "ner_chunk": "Hyperlipidemia", "ner_label": "PROBLEM", "ner_confidence": "0.9989", "concept_code": "0", "resolution": "hyperlipidemia", "score": 0.9999998211860657, "all_codes": ["0", "0", "0", "0", 

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

